In [1]:
import numpy as np
import pandas as pd

In [30]:
df=pd.read_csv("./runs/runs.csv").drop(columns=["Start Time", "Duration", "Run ID", "Source Type", "Status"])
automl_df=df[["AutoML" in name for name in df["Name"]]]
automl_df[["Model","TargetMetric","Threshold","Treshold_type","Seed"]]=automl_df["Name"].str.split("_",expand=True)
for idx in automl_df.index:
    if automl_df.loc[idx,"Seed"] is None:
        automl_df.loc[idx,"Seed"]=automl_df.loc[idx,"Treshold_type"]
automl_df.drop(columns=["Treshold_type","TargetMetric"],inplace=True)
df =df[[not "AutoML"in name for name in df["Name"]]]
df[["Model","TargetMetric","Threshold","Seed"]]=df["Name"].str.split("_",expand=True)

C:\Users\z003hfnh\AppData\Local\Temp\ipykernel_32388\2083096303.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\z003hfnh\AppData\Local\Temp\ipykernel_32388\2083096303.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\z003hfnh\AppData\Local\Temp\ipykernel_32388\2083096303.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [31]:
df[["Model","TargetMetric","Threshold","Seed"]]=df["Name"].str.split("_",expand=True)

In [32]:
dummy_df=df[df["Name"]=="DummyModel"]
df=df[df["Name"]!="DummyModel"]

In [33]:
metric_columns= [c for c in df.columns if "Test" in c or "Eval" in c]

In [34]:
new_df_list=list()
for model in np.unique(df["Model"]):
    for metric in np.unique(df["TargetMetric"]):
        for threshold in np.unique(df["Threshold"]):
            filtered_df=df[df["Model"]==model]
            filtered_df=filtered_df[filtered_df["TargetMetric"]==metric]
            filtered_df=filtered_df[filtered_df["Threshold"]==threshold]
            calculation_dict=dict()
            calculation_dict["Model"]=model
            calculation_dict["TargetMetric"]=metric
            calculation_dict["Threshold"]=threshold
            for metric_column in metric_columns:
                calculation_dict[metric_column+"Std"]=np.std(filtered_df[metric_column])
                calculation_dict[metric_column+"Mean"]=np.mean(filtered_df[metric_column])
                calculation_dict[metric_column+"Max"]=np.max(filtered_df[metric_column])
                calculation_dict[metric_column+"Min"]=np.min(filtered_df[metric_column])
                calculation_dict[metric_column+"Median"]=np.median(filtered_df[metric_column])
                calculation_dict[metric_column+"PrintOut"]=str(np.round(np.mean(filtered_df[metric_column]),3)) +u"\u00B1"+str(np.round(np.std(filtered_df[metric_column]),3))
                if "Recall" in metric_column and metric_column[0]=="R":
                    calculation_dict["Slip"+metric_column[6:]+"Std"]=np.std(filtered_df[metric_column])
                    calculation_dict["Slip"+metric_column[6:]+"Mean"]=np.mean(filtered_df[metric_column])
                    calculation_dict["Slip"+metric_column[6:]+"PrintOut"]=str(np.round(1-np.mean(filtered_df[metric_column]),3)) +u"\u00B1"+str(np.round(np.std(filtered_df[metric_column]),3))
            new_df_list.append(calculation_dict)
    #         break
    #     break
    # break
eval_df=pd.DataFrame.from_records(new_df_list)

In [35]:
new_df_list=list()
for model in np.unique(dummy_df["Model"]):
    # for metric in np.unique(dummy_df["TargetMetric"]):
    # for threshold in np.unique(dummy_df["Threshold"]):
    filtered_df=dummy_df[dummy_df["Model"]==model]
    # filtered_df=filtered_df[filtered_df["TargetMetric"]==metric]
    # filtered_df=filtered_df[filtered_df["Threshold"]==threshold]
    calculation_dict=dict()
    calculation_dict["Model"]=model
    calculation_dict["TargetMetric"]=metric
    calculation_dict["Threshold"]=threshold
    for metric_column in metric_columns:
        calculation_dict[metric_column+"Std"]=np.std(filtered_df[metric_column])
        calculation_dict[metric_column+"Mean"]=np.mean(filtered_df[metric_column])
        calculation_dict[metric_column+"Max"]=np.max(filtered_df[metric_column])
        calculation_dict[metric_column+"Min"]=np.min(filtered_df[metric_column])
        calculation_dict[metric_column+"Median"]=np.median(filtered_df[metric_column])
        calculation_dict[metric_column+"PrintOut"]=str(np.round(np.mean(filtered_df[metric_column]),3)) +u"\u00B1"+str(np.round(np.std(filtered_df[metric_column]),3))
        if "Recall" in metric_column and metric_column[0]=="R":
            calculation_dict["Slip"+metric_column[6:]+"Std"]=np.std(filtered_df[metric_column])
            calculation_dict["Slip"+metric_column[6:]+"Mean"]=np.mean(filtered_df[metric_column])
            calculation_dict["Slip"+metric_column[6:]+"PrintOut"]=str(np.round(1-np.mean(filtered_df[metric_column]),3)) +u"\u00B1"+str(np.round(np.std(filtered_df[metric_column]),3))
    new_df_list.append(calculation_dict)
#         break
#     break
# break
dummy_eval_df=pd.DataFrame.from_records(new_df_list)

In [36]:
tempdf=eval_df[["Model","TargetMetric","Threshold"]+[c for c in eval_df.columns if "PrintOut" in c and "Test" in c]]
tempdf=tempdf[tempdf["TargetMetric"]=="PrecisionRecallAUC"]
tempdf[tempdf["Threshold"]=="AdaptedThreshold"][["Model","AccuracyTestPrintOut","F1ScoreTestPrintOut","PrecisionRecallAUCTestPrintOut","YoudenIndexTestPrintOut"]]

,Model,AccuracyTestPrintOut,F1ScoreTestPrintOut,PrecisionRecallAUCTestPrintOut,YoudenIndexTestPrintOut
2,BRFC,0.964±0.005,0.299±0.029,0.713±0.02,0.919±0.013
6,RFC,0.983±0.005,0.485±0.094,0.842±0.016,0.948±0.01
10,XGBoost,0.993±0.005,0.699±0.124,0.909±0.015,0.959±0.013
14,kNN,0.981±0.011,0.473±0.146,0.739±0.023,0.879±0.036


In [37]:
tempdf=eval_df[["Model","TargetMetric","Threshold"]+[c for c in eval_df.columns if "PrintOut" in c and "Test" in c]]
tempdf=tempdf[tempdf["TargetMetric"]=="ConstrainedAUC"]
tempdf[tempdf["Threshold"]=="AdaptedThreshold"][["Model","VolumeReductionAtRecallTestPrintOut","ConstrainedVolumeReductionTestPrintOut","ConstrainedAUCTestPrintOut"]]

,Model,VolumeReductionAtRecallTestPrintOut,ConstrainedVolumeReductionTestPrintOut,ConstrainedAUCTestPrintOut
0,BRFC,0.796±0.064,0.423±0.396,0.305±0.19
4,RFC,0.901±0.042,0.611±0.416,0.5±0.169
8,XGBoost,0.925±0.038,0.16±0.324,0.626±0.12
12,kNN,0.475±0.344,0.227±0.339,0.079±0.091


In [38]:
tempdf=eval_df[["Model","TargetMetric","Threshold"]+[c for c in eval_df.columns if "PrintOut" in c and "Test" in c]]
# tempdf=tempdf[tempdf["TargetMetric"]=="ConstrainedAUC"]
tempdf[tempdf["Threshold"]=="AdaptedThreshold"][["Model","TargetMetric","SlipTestPrintOut","RecallTestPrintOut","VolumeReductionTest PrintOut"]]#, "AccuracyTestPrintOut","F1ScoreTestPrintOut","PrecisionRecallAUCTestPrintOut","YoudenIndexTestPrintOut","VolumeReductionAtRecallTestPrintOut","ConstrainedVolumeReductionTestPrintOut","ConstrainedAUCTestPrintOut"]]

,Model,TargetMetric,SlipTestPrintOut,RecallTestPrintOut,VolumeReductionTest PrintOut
0,BRFC,ConstrainedAUC,0.008±0.006,0.992±0.006,0.797±0.03
2,BRFC,PrecisionRecallAUC,0.052±0.015,0.948±0.015,0.964±0.005
4,RFC,ConstrainedAUC,0.01±0.012,0.99±0.012,0.897±0.039
6,RFC,PrecisionRecallAUC,0.057±0.057,0.943±0.057,0.983±0.005
8,XGBoost,ConstrainedAUC,0.015±0.021,0.985±0.021,0.596±0.429
10,XGBoost,PrecisionRecallAUC,0.078±0.043,0.922±0.043,0.993±0.005
12,kNN,ConstrainedAUC,0.014±0.018,0.986±0.018,0.555±0.311
14,kNN,PrecisionRecallAUC,0.15±0.139,0.85±0.139,0.982±0.012


In [39]:
tempdf=eval_df[["Model","TargetMetric","Threshold"]+[c for c in eval_df.columns if "PrintOut" in c and "Eval" in c and "VolumeReductionE" in c and c[0]=="V"]]
tempdf=tempdf[tempdf["TargetMetric"]=="ConstrainedAUC"]
tempdf[tempdf["Threshold"]=="AdaptedThreshold"]#[["Model","TargetMetric","SlipTestPrintOut","RecallTestPrintOut","VolumeReductionTest PrintOut"]]#, "AccuracyTestPrintOut","F1ScoreTestPrintOut","PrecisionRecallAUCTestPrintOut","YoudenIndexTestPrintOut","VolumeReductionAtRecallTestPrintOut","ConstrainedVolumeReductionTestPrintOut","ConstrainedAUCTestPrintOut"]]

,Model,TargetMetric,Threshold,VolumeReductionEval0 PrintOut,VolumeReductionEval1 PrintOut,VolumeReductionEval2 PrintOut,VolumeReductionEval3 PrintOut,VolumeReductionEval4 PrintOut
0,BRFC,ConstrainedAUC,AdaptedThreshold,0.675±0.045,0.487±0.066,0.653±0.066,0.78±0.043,0.415±0.057
4,RFC,ConstrainedAUC,AdaptedThreshold,0.667±0.084,0.524±0.122,0.654±0.092,0.76±0.083,0.397±0.123
8,XGBoost,ConstrainedAUC,AdaptedThreshold,0.598±0.429,0.53±0.395,0.601±0.43,0.615±0.438,0.537±0.393
12,kNN,ConstrainedAUC,AdaptedThreshold,0.473±0.267,0.36±0.223,0.465±0.268,0.526±0.301,0.279±0.215


In [40]:
tdf=df.copy()
tdf=tdf[tdf["Model"]=="RFC"]
tdf=tdf[tdf["TargetMetric"]=="ConstrainedAUC"]
tdf=tdf[tdf["AdaptThreshold"]==True]
tdf[[c for c in metric_columns if "Test" in c]+["Seed"]].sort_values(by="RecallTest")

,AccuracyTest,ConstrainedAUCTest,ConstrainedVolumeReductionTest,F1ScoreTest,PerfectThresholdTest,PrecisionRecallAUCTest,RecallTest,VolumeReductionAtRecallTest,VolumeReductionTest,YoudenIndexTest,Seed
202,0.966408,0.320330,-0.041429,0.309846,0.042857,0.682190,0.948571,0.827167,0.966551,0.918190,42
82,0.874989,0.421015,-0.001429,0.111685,0.010076,0.791234,0.988571,0.855282,0.874078,0.927412,911
106,0.932407,0.337147,-0.001429,0.188659,0.011152,0.843557,0.988571,0.915495,0.931957,0.954747,525600
170,0.910035,0.321020,-0.001429,0.148721,0.019231,0.806064,0.988571,0.830670,0.909405,0.940292,300
34,0.856364,0.279056,0.855282,0.098889,0.058511,0.711181,0.991429,0.875132,0.855282,0.927728,42
42,0.887799,0.611009,0.886968,0.123181,0.026201,0.806242,0.991429,0.914740,0.886968,0.942214,101
50,0.932656,0.510936,0.932186,0.189669,0.023346,0.825381,0.991429,0.932186,0.932186,0.952444,300
66,0.908604,0.467775,0.907917,0.147458,0.035000,0.815317,0.994286,0.947296,0.907917,0.953849,404
90,0.925048,0.811837,0.924493,0.174174,0.029240,0.858816,0.994286,0.935643,0.924493,0.958131,1337
98,0.892637,0.491990,0.891822,0.128342,0.008798,0.800515,0.994286,0.891822,0.891822,0.932521,58008


In [41]:
new_df_list=list()
for model in np.unique(automl_df["Model"]):
    for threshold in np.unique(automl_df["Threshold"]):
        filtered_df=automl_df[automl_df["Model"]==model]
        filtered_df=filtered_df[filtered_df["Threshold"]==threshold]
        calculation_dict=dict()
        calculation_dict["Model"]=model
        calculation_dict["Threshold"]=threshold
        for metric_column in metric_columns:
            calculation_dict[metric_column+"Std"]=np.std(filtered_df[metric_column])
            calculation_dict[metric_column+"Mean"]=np.mean(filtered_df[metric_column])
            calculation_dict[metric_column+"Max"]=np.max(filtered_df[metric_column])
            calculation_dict[metric_column+"Min"]=np.min(filtered_df[metric_column])
            calculation_dict[metric_column+"Median"]=np.median(filtered_df[metric_column])
            calculation_dict[metric_column+"PrintOut"]=str(np.round(np.mean(filtered_df[metric_column]),3)) +u"\u00B1"+str(np.round(np.std(filtered_df[metric_column]),3))
            if "Recall" in metric_column and metric_column[0]=="R":
                calculation_dict["Slip"+metric_column[6:]+"Std"]=np.std(filtered_df[metric_column])
                calculation_dict["Slip"+metric_column[6:]+"Mean"]=np.mean(filtered_df[metric_column])
                calculation_dict["Slip"+metric_column[6:]+"PrintOut"]=str(np.round(1-np.mean(filtered_df[metric_column]),3)) +u"\u00B1"+str(np.round(np.std(filtered_df[metric_column]),3))
        new_df_list.append(calculation_dict)
automl_eval_df=pd.DataFrame.from_records(new_df_list)

In [42]:
automl_eval_df

,Model,Threshold,AccuracyEval0Std,AccuracyEval0Mean,AccuracyEval0Max,AccuracyEval0Min,AccuracyEval0Median,AccuracyEval0PrintOut,AccuracyEval1Std,AccuracyEval1Mean,...,YoudenIndexEval4Max,YoudenIndexEval4Min,YoudenIndexEval4Median,YoudenIndexEval4PrintOut,YoudenIndexTestStd,YoudenIndexTestMean,YoudenIndexTestMax,YoudenIndexTestMin,YoudenIndexTestMedian,YoudenIndexTestPrintOut
0,AutoML,Default,0.000361,0.995905,0.996411,0.995389,0.995912,0.996±0.0,0.002946,0.996439,...,0.597711,0.511249,0.536132,0.543±0.022,0.005648,0.943565,0.952934,0.934546,0.942471,0.944±0.006
1,AutoML,VRS,0.199498,0.758516,0.993527,0.522588,0.704829,0.759±0.199,0.294015,0.650512,...,0.597711,0.511249,0.536132,0.543±0.022,0.005648,0.943565,0.952934,0.934546,0.942471,0.944±0.006
2,AutoML,YS,0.088701,0.888428,0.994458,0.778295,0.843380,0.888±0.089,0.106732,0.866948,...,0.597711,0.511249,0.536132,0.543±0.022,0.005648,0.943565,0.952934,0.934546,0.942471,0.944±0.006


In [43]:
tempdf=eval_df[["Model","TargetMetric","Threshold"]+[c for c in eval_df.columns if "PrintOut" in c and "Test" in c]]
tempdf[tempdf["Threshold"]=="AdaptedThreshold"][["Model","TargetMetric","SlipTestPrintOut","RecallTestPrintOut","VolumeReductionTest PrintOut"]]#, 

,Model,TargetMetric,SlipTestPrintOut,RecallTestPrintOut,VolumeReductionTest PrintOut
0,BRFC,ConstrainedAUC,0.008±0.006,0.992±0.006,0.797±0.03
2,BRFC,PrecisionRecallAUC,0.052±0.015,0.948±0.015,0.964±0.005
4,RFC,ConstrainedAUC,0.01±0.012,0.99±0.012,0.897±0.039
6,RFC,PrecisionRecallAUC,0.057±0.057,0.943±0.057,0.983±0.005
8,XGBoost,ConstrainedAUC,0.015±0.021,0.985±0.021,0.596±0.429
10,XGBoost,PrecisionRecallAUC,0.078±0.043,0.922±0.043,0.993±0.005
12,kNN,ConstrainedAUC,0.014±0.018,0.986±0.018,0.555±0.311
14,kNN,PrecisionRecallAUC,0.15±0.139,0.85±0.139,0.982±0.012


In [44]:
tempdf=automl_eval_df[["Model","Threshold"]+[c for c in automl_eval_df.columns if "PrintOut" in c and "Test" in c]]
# tempdf=tempdf[tempdf["TargetMetric"]=="ConstrainedAUC"]
tempdf[["Model","Threshold","AccuracyTestPrintOut","F1ScoreTestPrintOut","PrecisionRecallAUCTestPrintOut","YoudenIndexTestPrintOut","VolumeReductionAtRecallTestPrintOut","ConstrainedVolumeReductionTestPrintOut","ConstrainedAUCTestPrintOut","SlipTestPrintOut","VolumeReductionTest PrintOut"]]

,Model,Threshold,AccuracyTestPrintOut,F1ScoreTestPrintOut,PrecisionRecallAUCTestPrintOut,YoudenIndexTestPrintOut,VolumeReductionAtRecallTestPrintOut,ConstrainedVolumeReductionTestPrintOut,ConstrainedAUCTestPrintOut,SlipTestPrintOut,VolumeReductionTest PrintOut
0,AutoML,Default,0.997±0.0,0.763±0.027,0.866±0.016,0.944±0.006,0.906±0.021,-0.336±0.039,0.497±0.136,0.346±0.039,1.0±0.0
1,AutoML,VRS,0.949±0.049,0.413±0.283,0.866±0.016,0.944±0.006,0.906±0.021,0.103±0.402,0.497±0.136,0.083±0.095,0.949±0.049
2,AutoML,YS,0.987±0.009,0.581±0.176,0.866±0.016,0.944±0.006,0.906±0.021,-0.103±0.089,0.497±0.136,0.113±0.089,0.988±0.01


In [45]:
number_dict=dict(RFC="3",BRFC="1",kNN="2",XGBoost="4")
eval_df["Letter"]= ["S" if target=="PrecisionRecallAUC" else "D" for target in eval_df["TargetMetric"]]
eval_df["Number"]= [number_dict[model] for model in eval_df["Model"]]
eval_df["Symbol"]=eval_df["Letter"]+eval_df["Number"]

automl_eval_df["Symbol"]= ["A1" if target=="YS" else "A2" for target in automl_eval_df["Threshold"]]
dummy_eval_df["Symbol"]="DC1"

plot_df=pd.concat([eval_df,automl_eval_df],ignore_index=True)
mask=[element=="AdaptedThreshold" or element=="VRS" or element=="YS" for element in plot_df["Threshold"]]
plot_df=plot_df[mask]
plot_df=pd.concat([plot_df,dummy_eval_df],ignore_index=True)

mask=["Model", "TargetMetric","Threshold","Symbol"] + [c for c in plot_df.columns if "Test" in c and ("Mean" in c or "Std" in c)]
plot_df=plot_df[mask]

mean_cols=['AccuracyTestMean',
 'F1ScoreTestMean',
 'PrecisionRecallAUCTestMean',
 'YoudenIndexTestMean',
 'ConstrainedVolumeReductionTestMean',
 'VolumeReductionAtRecallTestMean',
 'ConstrainedAUCTestMean',
 'SlipTestMean',
 'VolumeReductionTest Mean',
 ]
std_cols=['AccuracyTestStd',
 'F1ScoreTestStd',
 'PrecisionRecallAUCTestStd',
 'ConstrainedVolumeReductionTestStd',
 'VolumeReductionAtRecallTestStd',
 'ConstrainedAUCTestStd',
 'SlipTestStd',
 'VolumeReductionTest Std',
 'YoudenIndexTestStd']
labels=['Accuracy',
 'F1-Score',
 'PRC',
 'Youden Score',
 'cV',
 'V@S',
 'cAUC',
 'Slip rate',
 'Volume Reduction']
plot_df

,Model,TargetMetric,Threshold,Symbol,AccuracyTestStd,AccuracyTestMean,ConstrainedAUCTestStd,ConstrainedAUCTestMean,ConstrainedVolumeReductionTestStd,ConstrainedVolumeReductionTestMean,...,RecallTestStd,RecallTestMean,SlipTestStd,SlipTestMean,VolumeReductionAtRecallTestStd,VolumeReductionAtRecallTestMean,VolumeReductionTest Std,VolumeReductionTest Mean,YoudenIndexTestStd,YoudenIndexTestMean
0,BRFC,ConstrainedAUC,AdaptedThreshold,D1,0.029930,0.798763,0.189591,0.304741,0.396085,0.423342,...,0.005873,0.991648,0.005873,0.991648,0.063589,0.795540,0.030189,0.797217,0.012464,0.917880
1,BRFC,PrecisionRecallAUC,AdaptedThreshold,S1,0.005142,0.964117,0.185472,0.306508,0.015042,-0.042088,...,0.015042,0.947912,0.015042,0.947912,0.071811,0.803696,0.005236,0.964247,0.012902,0.919143
2,RFC,ConstrainedAUC,AdaptedThreshold,D3,0.038321,0.898021,0.168868,0.499620,0.415851,0.610540,...,0.012355,0.989670,0.012355,0.989670,0.041657,0.900762,0.038686,0.897287,0.012035,0.941540
3,RFC,PrecisionRecallAUC,AdaptedThreshold,S3,0.004957,0.983007,0.239163,0.338872,0.056751,-0.046923,...,0.056751,0.943077,0.056751,0.943077,0.328309,0.764847,0.005381,0.983327,0.010085,0.947779
4,XGBoost,ConstrainedAUC,AdaptedThreshold,D4,0.425193,0.599217,0.120315,0.626158,0.324008,0.160295,...,0.020725,0.985079,0.020725,0.985079,0.038133,0.925009,0.428704,0.596125,0.013087,0.952718
5,XGBoost,PrecisionRecallAUC,AdaptedThreshold,S4,0.004527,0.992805,0.151748,0.573840,0.043249,-0.068413,...,0.043249,0.921587,0.043249,0.921587,0.051289,0.923076,0.004849,0.993376,0.012972,0.958960
6,kNN,ConstrainedAUC,AdaptedThreshold,D2,0.308287,0.558412,0.090910,0.078582,0.338957,0.226791,...,0.017664,0.985714,0.017664,0.985714,0.343804,0.474615,0.310847,0.554988,0.030588,0.853252
7,kNN,PrecisionRecallAUC,AdaptedThreshold,S2,0.011422,0.981065,0.025709,-0.044172,0.139022,-0.140476,...,0.139022,0.849524,0.139022,0.849524,0.000000,0.000000,0.012309,0.982119,0.036004,0.878619
8,AutoML,NaN,VRS,A2,0.048533,0.948878,0.135632,0.496780,0.401529,0.102991,...,0.095456,0.917143,0.095456,0.917143,0.021467,0.906365,0.049495,0.949132,0.005648,0.943565
9,AutoML,NaN,YS,A1,0.008858,0.986813,0.135632,0.496780,0.089098,-0.102857,...,0.089098,0.887143,0.089098,0.887143,0.021467,0.906365,0.009562,0.987612,0.005648,0.943565


In [46]:
import plotly.express as px
import plotly.graph_objects as go
import sys
sys.path.insert(0,'..')
from plotly.subplots import make_subplots

In [47]:
fig=go.Figure()
for symbol in plot_df["Symbol"].unique():
    fig.add_trace(go.Bar(x=labels,y=plot_df[plot_df["Symbol"]==symbol][mean_cols].values.ravel(),name=symbol))#]plot_df[plot_df["Symbol"]==symbol][mean_cols]
fig.update_layout(legend=dict(y=1.04,x=0.05,orientation="h"),title="Performance comparison of all models",margin=dict(l=0,r=0,t=30,b=0),width=800,height=400, yaxis=dict(tickmode="linear",dtick=0.2))
fig.show()

In [48]:

fig=make_subplots(rows=3, cols=3,subplot_titles=labels)
i=0
j=1
for metric in mean_cols:
    # print(metric,(i//3)+1,(i)%3+1,"\n")
    if metric=="SlipTestMean":
        y=1-plot_df[metric]
    else:
        y=plot_df[metric]
    error_y=dict(array=plot_df[std_cols[i]].values)
    x=plot_df["Symbol"]
    fig.add_trace(go.Scatter(x=x, y=y,error_y=error_y, mode='markers',name=labels[i]), row=(i//3)+1, col=i%3+1)
    # fig.update_yaxes(range=[0,1.1], row=(i//3)+1, col=(i-1)%3+1)
    # fig.update_layout(margin=dict(b=0), row=(i//3)+1, col=(i-1)%3+1)
    i+=1
fig.update_layout(width=1000,height=600, margin=dict(l=0, r=0, t=20, b=0),showlegend=False)
fig.update_xaxes(title="Model reference")
fig.update_yaxes(title="Value")
fig.show()
fig.write_image("./figs/Model_comparison.svg")

In [49]:
time_df=eval_df[[symbol=="D1" or symbol=="D3" for symbol in eval_df["Symbol"]]]
time_df=time_df[time_df["Threshold"]=="AdaptedThreshold"]
cols=[c for c in time_df.columns if ("Mean" in c or "Std" in c) and ("Slip" in c or ("VolumeReduction" in c ))]# and  not "Recall" in c not "Constrained" in c and
time_df=time_df[["Model","TargetMetric", "Threshold","Symbol"]+cols]
time_df

,Model,TargetMetric,Threshold,Symbol,ConstrainedVolumeReductionEval0Std,ConstrainedVolumeReductionEval0Mean,ConstrainedVolumeReductionEval1Std,ConstrainedVolumeReductionEval1Mean,ConstrainedVolumeReductionEval2Std,ConstrainedVolumeReductionEval2Mean,...,VolumeReductionEval1 Std,VolumeReductionEval1 Mean,VolumeReductionEval2 Std,VolumeReductionEval2 Mean,VolumeReductionEval3 Std,VolumeReductionEval3 Mean,VolumeReductionEval4 Std,VolumeReductionEval4 Mean,VolumeReductionTest Std,VolumeReductionTest Mean
0,BRFC,ConstrainedAUC,AdaptedThreshold,D1,0.025344,-0.062874,0.015779,-0.043254,0.030544,-0.076404,...,0.066024,0.487191,0.066297,0.653039,0.043108,0.779705,0.057409,0.415436,0.030189,0.797217
4,RFC,ConstrainedAUC,AdaptedThreshold,D3,0.073786,-0.092227,0.025412,-0.135957,0.056394,-0.139968,...,0.122207,0.523554,0.091871,0.653741,0.083193,0.759576,0.123019,0.396749,0.038686,0.897287


In [50]:
slip_mean_cols=['SlipTestMean', 'SlipEval0Mean', 'SlipEval1Mean', 'SlipEval2Mean', 'SlipEval3Mean', 'SlipEval4Mean']
volume_mean_cols=['VolumeReductionTest Mean', 'VolumeReductionEval0 Mean', 'VolumeReductionEval1 Mean', 'VolumeReductionEval2 Mean', 'VolumeReductionEval3 Mean', 'VolumeReductionEval4 Mean']
slip_std_cols=['SlipTestStd', 'SlipEval0Std', 'SlipEval1Std', 'SlipEval2Std', 'SlipEval3Std', 'SlipEval4Std']
volume_std_cols=['VolumeReductionTest Std', 'VolumeReductionEval0 Std', 'VolumeReductionEval1 Std', 'VolumeReductionEval2 Std', 'VolumeReductionEval3 Std', 'VolumeReductionEval4 Std']
x_labels=["Test dataset", "Evaluation slice 1", "Evaluation slice 2", "Evaluation slice 3", "Evaluation slice 4", "Evaluation slice 5",]

In [51]:
fig=make_subplots(rows=2, cols=1,subplot_titles=["Slip rates","Volume reductions"])
colors=["#00D7A0","#00646E", "#EC6602"]
counter=0
for symbol in time_df['Symbol'].unique():
    fig.add_trace(go.Scatter(x=x_labels, y=1-time_df[time_df["Symbol"]==symbol][slip_mean_cols].values.ravel(),error_y=dict(array=time_df[time_df["Symbol"]==symbol][slip_std_cols].values.ravel()),marker=dict(color=colors[counter]), name=symbol, legendgroup=symbol,showlegend=True),row=1, col=1)
    fig.add_trace(go.Scatter(x=x_labels,y=time_df[time_df["Symbol"]==symbol][volume_mean_cols].values.ravel(),error_y=dict(array=time_df[time_df["Symbol"]==symbol][volume_std_cols].values.ravel()),marker=dict(color=colors[counter]), name=symbol, legendgroup=symbol,showlegend=False),row=2, col=1)
    counter+=1
fig.add_trace(go.Scatter(x=x_labels, y=[0.01]*6, name="Requirement", mode="lines",legendgroup="Requirement",marker=dict(color=colors[counter])),row=1, col=1)
fig.add_trace(go.Scatter(x=x_labels, y=[0.4]*6, name="Requirement", mode="lines",legendgroup="Requirement", showlegend=False,marker=dict(color=colors[counter])),row=2, col=1)
fig.update_layout(width=800,height=600, margin=dict(l=0, r=0, t=20, b=0), legend=dict(y=0.5,x=0.35,orientation="h"))
fig.update_xaxes(type='category',title="Evaluation dataset")
# fig.update_xaxes(linewidth=2, linecolor='black',showline=True,row=2,col=1)#yaxis=dict(tickmode="linear",dtick=0.2)
fig.update_yaxes(tickmode="linear",dtick=0.1,row=2,col=1)
fig.update_yaxes(title="Value",linewidth=2, linecolor='black',showline=True)

fig.show()
fig.write_image("./figs/Evaluation_timeline.svg")